In [1]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import psutil
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

from oceanoobsbrasil.db import GetData
from oceanoobsbrasil.utils import *

%load_ext autoreload
%autoreload 2

In [2]:
from oceanoobsbrasil.tides.santos_tide import TideSantos

In [3]:
t = TideSantos()

In [215]:
t.url = 'http://www.santospilots.com.br:8888'

In [4]:
t.get()

ilha das palmas


ProgrammingError: (psycopg2.errors.SyntaxError) ERRO:  erro de sintaxe em ou próximo a ")"
LINE 1: ... FROM data_stations WHERE true AND station_id in () AND date...
                                                             ^

[SQL: DELETE FROM data_stations WHERE true AND station_id in () AND date_time >= 'NaT']
(Background on this error at: https://sqlalche.me/e/14/f405)

In [15]:
df = pd.read_html(t.driver.page_source)[1]
df['HORA'] = pd.to_datetime((datetime.utcnow()-timedelta(hours=3)).strftime("%Y-%m-%d") + df['HORA'], format='%Y-%m-%d%H:%M')
df['HORA'] = df['HORA'] + timedelta(hours=3)

In [16]:
df.columns = ['date_time', 'water_level_pred', 'water_level']
df

,date_time,water_level_pred,water_level
0,2021-10-17 03:00:00,1.16,1.44
1,2021-10-17 04:00:00,1.24,1.44
2,2021-10-17 05:00:00,1.18,1.39
3,2021-10-17 06:00:00,0.95,1.18
4,2021-10-17 07:00:00,0.63,0.90
5,2021-10-17 08:00:00,0.33,0.57
6,2021-10-17 09:00:00,0.15,0.47
7,2021-10-17 10:00:00,0.13,0.43
8,2021-10-17 11:00:00,0.23,0.55
9,2021-10-17 12:00:00,0.43,0.69


In [17]:
df = df[df.date_time < datetime.utcnow()]

df['meteorological_tide'] = df['water_level'] - df['water_level_pred']

df.drop(columns='water_level_pred', inplace=True)

In [18]:
df

,date_time,water_level,meteorological_tide
0,2021-10-17 03:00:00,1.44,0.28
1,2021-10-17 04:00:00,1.44,0.20
2,2021-10-17 05:00:00,1.39,0.21
3,2021-10-17 06:00:00,1.18,0.23
4,2021-10-17 07:00:00,0.90,0.27
5,2021-10-17 08:00:00,0.57,0.24
6,2021-10-17 09:00:00,0.47,0.32
7,2021-10-17 10:00:00,0.43,0.30
8,2021-10-17 11:00:00,0.55,0.32
9,2021-10-17 12:00:00,0.69,0.26


In [9]:
pd.read_html(t.driver.page_source)[1]

,HORA,MARÉ PREVISTA,MARÉ REAL
0,00:00,1.16,1.44
1,01:00,1.24,1.44
2,02:00,1.18,1.39
3,03:00,0.95,1.18
4,04:00,0.63,0.90
5,05:00,0.33,0.57
6,06:00,0.15,0.47
7,07:00,0.13,0.43
8,08:00,0.23,0.55
9,09:00,0.43,0.69


In [5]:
t.result

,date_time,water_level,meteorological_tide,station_id


In [ ]:
driver.find_elements_by_xpath("//img[@alt='Ver Marégrafo']")

In [163]:
t.login()

you are already logged in


In [165]:
t.points_of_tide = t.driver.find_elements_by_xpath("//img[@alt='Ver Marégrafo']")

In [166]:
t.points_of_tide

[]

In [158]:
t.get()

you are already logged in
driver is running
chrome is still running, we can quit
driver has died


In [160]:
t.driver.find_elements_by_xpath("//img[@alt='Ver Marégrafo']")

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=43737): Max retries exceeded with url: /session/5d9c42293979817ca6ac8d795d32f6e0/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f381ef028e0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [159]:
t.points_of_tide

[]

In [129]:
site = t.url
user = t.user
pwd = t.pwd 
driver = t.driver

In [130]:
driver.get(site)

login_elem = driver.find_element_by_xpath('//*[@id="CustomerEmail"]')
login_elem.send_keys(user)

psw_elem = driver.find_element_by_xpath("//*[@id='CustomerPassword']")
psw_elem.send_keys(pwd)

login_bt = driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div[3]/form/div[2]/input")
login_bt.click()

In [131]:
imgs = driver.find_elements_by_xpath("//img[@alt='Ver Marégrafo']")

In [132]:
imgs[0].click()
time.sleep(3)

In [133]:
driver.switch_to.frame("fancybox-frame")
name = driver.find_element_by_class_name("name").text.lower()
station = t.stations[t.stations.url == name]

In [134]:
df = pd.read_html(driver.page_source)[1]
df['HORA'] = pd.to_datetime(datetime.utcnow().strftime("%Y-%m-%d") + df['HORA'], format='%Y-%m-%d%H:%M')
df['HORA'] = df['HORA'] + timedelta(hours=3)

In [136]:
df.columns = ['date_time', 'water_level_pred', 'water_level']
df = df[df.date_time < datetime.utcnow()]
df

,date_time,water_level_pred,water_level
0,2021-10-17 03:00:00,1.16,1.44
1,2021-10-17 04:00:00,1.24,1.44
2,2021-10-17 05:00:00,1.18,1.39
3,2021-10-17 06:00:00,0.95,1.18
4,2021-10-17 07:00:00,0.63,0.90
5,2021-10-17 08:00:00,0.33,0.57
6,2021-10-17 09:00:00,0.15,0.47
7,2021-10-17 10:00:00,0.13,0.43
8,2021-10-17 11:00:00,0.23,0.55
9,2021-10-17 12:00:00,0.43,0.69


In [138]:
xpath_close_box = '//*[@id="fancybox-close"]'
driver.switch_to.default_content()
driver.find_element_by_xpath(xpath_close_box).click()
time.sleep(2)

In [139]:
imgs[1].click()

In [140]:
driver.switch_to.frame("fancybox-frame")
name = driver.find_element_by_class_name("name").text.lower()
station = t.stations[t.stations.url == name]

In [141]:
name

'praticagem'

In [46]:
t.station
pd.read_html(driver.page_source)[1]

In [48]:
soup=BeautifulSoup(driver.page_source, 'html.parser')

In [79]:
name = driver.find_element_by_class_name("name").text.lower()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".name"}
  (Session info: headless chrome=93.0.4577.63)


In [61]:
pd.read_html(driver.page_source)[1]

,HORA,MARÉ PREVISTA,MARÉ REAL
0,00:00,1.16,1.44
1,01:00,1.24,1.44
2,02:00,1.18,1.39
3,03:00,0.95,1.18
4,04:00,0.63,0.90
5,05:00,0.33,0.57
6,06:00,0.15,0.47
7,07:00,0.13,0.43
8,08:00,0.23,0.55
9,09:00,0.43,0.69


In [ ]:

l=soup.find_all(attrs={'id': 'Box01_arrow'})
x=l[0].path.attrs
x2=x['transform']

In [106]:
from oceanoobsbrasil.tides.ilha_fiscal_tide import IlhaFiscal

In [109]:
h = IlhaFiscal()

In [110]:
h.get()

driver is running
chrome is still running, we can quit
driver has died


In [111]:
h.result

,date_time,water_level,station_id
3,2021-10-17 10:40:00,2.55,904
4,2021-10-17 10:35:00,2.54,904
5,2021-10-17 10:30:00,2.56,904
6,2021-10-17 10:25:00,2.57,904
7,2021-10-17 10:20:00,2.47,904
...,...,...,...
197,2021-10-16 18:30:00,2.17,904
198,2021-10-16 18:25:00,2.12,904
199,2021-10-16 18:20:00,2.15,904
200,2021-10-16 18:15:00,2.14,904
